### Broyden's Method
This code implements Quasi-Newton Broyden's Method to approximate the solution of the nonlinear system $\mathbf{F}(\mathbf{x}) = \mathbf{0}$ where $\mathbf{x} = (x_1, x_2)^{\top}$. We must choose an initial guess/approximation $\mathbf{x}^{(0)}$ and calculate the Jacobian $\mathbf{J}(\mathbf{x}^{(0)})$ for the first step of the method. For subsequent iterations, we will perform rank-one updates to the Jacobian using the Sherman-Morrison Formula.

This algorithm is implemented according to the pseudocode given in Algorithm 10.2 of *Numerical Analysis* (10th Edition) by Burden and Faires. The code below provides the solution to 11.3.5(b) of the same book.

In [163]:
# Imports
import numpy as np
from numpy.linalg import inv
import pandas as pd
import math

# For more decimal places
pd.set_option("display.precision", 7)

In [164]:
# Functions
f1 = lambda x1, x2: 5*x1**2 - x2**2
f2 = lambda x1, x2: x2 - 0.25*(np.sin(x1) + np.cos(x2))

# Initial guess
x = np.matrix([[0.1], [0.3]])
# Tolerance
TOL = 10**(-6)

# Arrays for approximations for each iteration
x1k = np.array(x[0,0])
x2k = np.array(x[1,0])

In [165]:
# Partial derivatives (for step 1 Jacobian)
f1_x1 = lambda x1, x2: 10*x1
f1_x2 = lambda x1, x2: -2*x2
f2_x1 = lambda x1, x2: -1/4*np.cos(x1)
f2_x2 = lambda x1, x2: 1 + 1/4*np.sin(x2)

In [166]:
# Defining Jacobian
def Jac(x1, x2, inverse=0):
  mat = np.matrix([[f1_x1(x1,x2), f1_x2(x1,x2)], [f2_x1(x1,x2), f2_x2(x1,x2)]])
  if inverse == 0:
    return mat
  if inverse == 1:
    return inv(mat)

In [167]:
# Defining vector-valued function
def F(x1, x2):
  return np.matrix([[f1(x1,x2)], [f2(x1,x2)]])

In [168]:
# Defining l-max norm for a 2D vector
def norm(x):
  return max(abs(x[0,0]), abs(x[1,0]))

In [169]:
# Step 1
A0 = Jac(x[0,0], x[1,0])
v = F(x[0,0], x[1,0])

# Step 2
A = inv(A0)

# Step 3
s = np.matmul(-A, v)
x = x + s
k = 2

In [170]:
# Step 4
while True:

  # Step 5
  w = v
  v = F(x[0,0], x[1,0])
  y = v - w

  # Step 6
  z = np.matmul(-A, y)

  # Step 7
  p = np.matmul(-s.T, z)

  # Step 8
  ut = np.matmul(s.T, A)

  # Step 9
  A = A + np.multiply(1/p, np.matmul(s+z, ut))

  # Step 10
  s = np.matmul(-A, v)

  # Step 11
  x = x + s

  # Step 12
  if norm(s) < TOL:
    print(f'The procedure was successful after {k} iterations.')
    break

  # Step 13 (next iteration)
  k = k + 1

  # Appending approximations to array for output
  x1k = np.append(x1k, x[0,0])
  x2k = np.append(x2k, x[1,0])

The procedure was successful after 5 iterations.


In [171]:
# Output
df = pd.DataFrame({'x_1^(k)': x1k, 'x_2^(k)': x2k,})
df

,x_1^(k),x_2^(k)
0,0.1000000,0.3000000
1,0.1208222,0.2710125
2,0.1212378,0.2711043
3,0.1212419,0.2711052
